In [ ]:
import socket
import threading
import json
import sys

from time import time, sleep
import copy

from collections import OrderedDict


HOST = '192.168.0.140'  # Standard loopback interface address (localhost)
PORT = 65432 # Port to listen on (non-privileged ports are > 1023)
DATA_BUFF = 4096
OK = "Ok."
OBJECT_LIFETIME = 20 # Lifetime of the object (in seconds)
EXIT_THRESHOLD = 10 # Exit thread if didn't receive anything for X seconds

In [ ]:
class MsgObj():
    def __init__(self, obj_id, timestamp, msg, active = True, lastRead = 0):
        self.id = obj_id,
        self.active = active # Only active objects are being sent
        self.lastRead = lastRead # When the object was read last time
        self.timestamp = timestamp
        self.msg = msg

    def dumpObj(self):
        print ('Id: {}, Timestamp: {}, Msg: {}'.format(self.id, self.timestamp, self.msg))
        
        
class MsgObjects():
    def __init__(self):
        self.objects = OrderedDict()
    
    def first(self):
        if len(self.objects) == 0:
            return None
        return next(self.objects)
    
    def sort_objects(self):
        self.objects = OrderedDict(sorted(
            self.objects.items(), key=lambda item: item[1].lastRead))
        
    def push_object(self, msg, force_sort = False):
        if isinstance(msg, str):
            msg_dict = json.loads(msg)
        elif isinstance(msg, dict):
            msg_dict = msg
            msg = json.dumps(msg_dict)
            
        if 'id' in msg_dict:
            objid = msg_dict['id']
            if 'secMark' in msg_dict:
                timestamp = msg_dict['secMark']
            else:
                timestamp = time()
            if objid in self.objects.keys():
                self.objects[objid].msg = msg
                self.objects[objid].timestamp = timestamp
                self.objects[objid].active = True
            else:
                obj = MsgObj(objid, timestamp, msg)
                self.objects[obj.id] = obj
        else:
            return 1 # Error - cannot push the object
        
        if force_sort:
            self.sort_objects()
        return 0

    def pull_object(self, force_sort = False):
        if len(self.objects) == 0:
            return None
        if force_sort:
            self.sort_objects()
        key, obj = self.objects.popitem(last = False)
        
        # Recurrence. If our object is too old, let's take the next one. 
        # Current object is already removed from the list by popitem
        if time() - obj.timestamp > OBJECT_LIFETIME:
            return (self.pull_object(False))
        
        
        self.objects[key] = obj
        if not obj.active:
            return None
        obj.lastRead = time()
        obj.active = False
        return obj
    
    
    def pull_objects(force_sort = False):
        if force_sort:
            self.sort_objects()
        ret = copy.deepcopy(self.objects)
        self.objects = OrderedDict()
        return ret
    
    def pull_bsm(self, force_sort = False):
        obj = self.pull_object(force_sort)
        if obj is None:
            return '{"msg" : ""}'
        return obj.msg
    
    def get_bsms(self, force_sort = False):
        """
        Returns bsm msgs for all objects
        Arguments: force_sort - if true, then objects are sorted first (default = False)
        Returns:   list with bsm msgs in json format
        """
        if force_sort:
            self.sort_objects()
        return [x.msg for x in self.objects.values()]
        
    

In [ ]:



buff = None
counter = 0

msgObjects = MsgObjects()




def push_data(request):
    global msgObjects
    json_msgs = json.loads(request['msg'])
    #print ("json_msgs:", type(json_msgs), json_msgs)
    if isinstance(json_msgs, dict):
        json_msgs = [json_msgs]
    ret = sum([msgObjects.push_object(json_msg) for json_msg in json_msgs])
    msgObjects.sort_objects()
    if ret > 0:
        return "ERROR: Failed to push {} / {} objects.".format(ret, len(json_msgs))
    return OK
        
def pull_data(conn, request):
    myMsg = msgObjects.pull_bsm()
    print ("PULL:", myMsg)
    myMsg = str.encode(myMsg)
    conn.sendall(myMsg)         

def check_data(conn, request):
    myMsg = { 'msgs' : msgObjects.get_bsms()}
    myMsg = json.dumps(myMsg)
    myMsg = str.encode(myMsg)
    conn.sendall(myMsg)         
        
    
def connection(nr, conn, addr):
    last_received = time()
    global buff
    global counter
    print ("Connection nr: {}, c: {}, addr: {}".format(nr, conn, addr))
    while True:
        data = conn.recv(DATA_BUFF)
        if addr[0] != HOST:
            print("Conn {}, addr: {}, Data received: {}".format(nr, addr, data))
        if len(data) == 0:
            if time() - last_received < EXIT_THRESHOLD: 
                continue
            else: # End thread if not conected for a while
                print ("\nDisconnecting session...")
                break
        last_received = time()
        request = json.loads(data)
        if not "mode" in request:
            conn.sendall(str.encode('ERROR: No "mode" in request.'))
            continue
        if request['mode'] == 'push':
            ret = push_data(request)
            conn.sendall(str.encode(ret))
        elif request['mode'] == 'pull':
            pull_data(conn, request)
        elif request['mode'] == 'check':
            check_data(conn, request)
        else:
            conn.sendall(str.encode('ERROR: Value for "mode" unknown.'))
            
connr = 0

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((HOST, PORT))
    s.listen()
    while True:
        conn, addr = s.accept()
        connr += 1
        t = threading.Thread(target=connection, args=(connr, conn, addr))
        t.start()

In [ ]:
s.close()

In [ ]:
for k, obj in msgObjects.objects.items():
    obj.dumpObj()

In [ ]:
msgObjects.objects

In [ ]:
dic = {}
for i in range(1, 5):
    dic[i] = "A" * i
dic = OrderedDict(dic)
dic

In [ ]:
dic.items()


In [ ]:
k, o = dic.popitem(last = False)
dic[k] = o
print (o)
dic

In [ ]:
dic


In [ ]:
dic([1, 2])

In [ ]:
print (time())
sleep(2)
print (time())

In [ ]:
objects.get_bsms()

In [ ]:
objects

In [ ]:
for x in msgObjects.objects:
    print (x, msgObjects.objects[x], msgObjects.objects[x].msg)

In [ ]:
msgObjects.get_bsms()

In [ ]:
for a in msgObjects.objects.values():
    print (a.msg)

In [ ]:
a = {'msgs' : msgObjects.get_bsms()}
a = json.dumps(a)
a